# Facial Emotion Recognition

## Import libraries

In [2]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.layers import Conv2D, Dense, BatchNormalization, Activation, Dropout, MaxPooling2D, Flatten
from keras.optimizers import Adam, RMSprop, SGD
from keras import regularizers
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau
import datetime
import matplotlib.pyplot as plt
from keras.utils import plot_model

In [3]:
train_dir = '../input/fer2013/train/'
test_dir = '../input/fer2013/test/'

img_l, img_w = 48, 48
num_classes = 7

def count_exp(path, set_):
    dict_ = {}
    for x in os.listdir(path):
        dir_ = path + x
        dict_[x] = len(os.listdir(dir_))
    df = pd.DataFrame(dict_, index=[set_])
    return df
train_count = count_exp(train_dir, 'train')
test_count = count_exp(test_dir, 'test')
print(train_count)
print(test_count)

### Graphical representation of training data

In [4]:
train_count.transpose().plot(kind='bar', figsize=(12,10))

### Graphical representation of testing data 

In [5]:
test_count.transpose().plot(kind='bar',figsize=(12, 10))

### Display sample images

In [6]:
plt.figure(figsize=(14,22))
i = 1
for x in os.listdir(train_dir):
    img = load_img((train_dir + x +'/'+ os.listdir(train_dir + x)[1]))
    plt.subplot(1,7,i)
    plt.imshow(img)
    plt.title(x)
    plt.axis('off')
    i += 1
plt.show()

In [7]:
image_size= 48 # image is a 48*48 grid of pixels
batch_size= 64 # number of images to process at a time
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range=0.3,
                                   horizontal_flip=True)

training_set = train_datagen.flow_from_directory(train_dir,
                                                batch_size=64,
                                                target_size=(image_size, image_size),
                                                shuffle=True,
                                                color_mode='grayscale',
                                                class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(test_dir,
                                                batch_size=64,
                                                target_size=(image_size, image_size),
                                                shuffle=True,
                                                color_mode='grayscale',
                                                class_mode='categorical')

In [8]:
training_set.class_indices

## Defining Model

In [9]:
def fer_model(input_size, num_classes=7):
    fer_model = tf.keras.models.Sequential()
    fer_model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(img_l, img_w,1)))
    fer_model.add(Conv2D(64,(3,3), padding='same', activation='relu' ))
    fer_model.add(BatchNormalization())
    fer_model.add(Activation('relu'))
    fer_model.add(MaxPooling2D(pool_size=(2, 2)))
    fer_model.add(Dropout(0.25))


    fer_model.add(Conv2D(128,(5,5), padding='same', activation='relu'))
    fer_model.add(BatchNormalization())
    fer_model.add(Activation('relu'))
    fer_model.add(MaxPooling2D(pool_size=(2, 2)))
    fer_model.add(Dropout(0.25))
    

    fer_model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    fer_model.add(BatchNormalization())
    fer_model.add(Activation('relu'))
    fer_model.add(MaxPooling2D(pool_size=(2, 2)))
    fer_model.add(Dropout(0.25))


    fer_model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    fer_model.add(BatchNormalization())
    fer_model.add(Activation('relu'))
    fer_model.add(MaxPooling2D(pool_size=(2, 2)))
    fer_model.add(Dropout(0.25))

    fer_model.add(Flatten())

    fer_model.add(Dense(256))
    fer_model.add(BatchNormalization())
    fer_model.add(Activation('relu'))
    fer_model.add(Dropout(0.25))
    
    
    fer_model.add(Dense(512))
    fer_model.add(BatchNormalization())
    fer_model.add(Activation('relu'))
    fer_model.add(Dropout(0.25))

    fer_model.add(Dense(num_classes, activation = 'softmax'))
    optimizer = Adam(lr = 0.0001 , decay = 1e-6)
    fer_model.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    fer_model.summary
    return fer_model

In [10]:
fer_model = fer_model((img_l,img_w,1), num_classes)
fer_model.summary()

In [11]:
plot_model(fer_model, to_file='fer_model.png', show_shapes=True, show_layer_names=True)

## Training the model

In [12]:
steps_per_epoch = training_set.n // training_set.batch_size
validation_steps = test_set.n // test_set.batch_size

# Checkpointing

In [13]:
chkpt_path = 'fer_model.h5'
log_dir = "checkpoint/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint = ModelCheckpoint(filepath=chkpt_path,
                             save_best_only=True,
                             verbose=1,
                             mode='min',
                             moniter='val_loss')

earlystop = EarlyStopping(monitor='val_loss', 
                          min_delta=0, 
                          patience=3, 
                          verbose=1, 
                          restore_best_weights=True)
                        
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2, 
                              patience=6, 
                              verbose=1, 
                              min_delta=0.0001)


tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
csv_logger = CSVLogger('training.log')

callback = [checkpoint, reduce_lr, csv_logger]

In [14]:
fer_model_hist = fer_model.fit(x=training_set,
                 validation_data=test_set,
                 epochs=100,
                 callbacks=callback,
                 steps_per_epoch=steps_per_epoch,
                 validation_steps=validation_steps)

## Loss and Accuracy plot

In [15]:
plt.figure(figsize=(14,5))
plt.subplot(1,2,2)
plt.plot(fer_model_hist.history['accuracy'])
plt.plot(fer_model_hist.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(1,2,1)
plt.plot(fer_model_hist.history['loss'])
plt.plot(fer_model_hist.history['val_loss'])
plt.title('model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Model evaluation

In [16]:
def model_data(model):
    train_loss, train_accu = fer_model.evaluate(training_set)
    test_loss, test_accu = fer_model.evaluate(test_set)
    return print("Train accuracy = {:.2f} , Validation accuracy = {:.2f}".format(train_accu*100, test_accu*100))

In [17]:
model_data(fer_model)

In [18]:
fer_model.save_weights('fer_model_bestweights.h5')

### Confusion Matrix and Classification on training set

In [20]:
y_pred = fer_model.predict(training_set)
y_pred = np.argmax(y_pred, axis=1)
class_labels = test_set.class_indices
class_labels = {value:key for key,value in class_labels.items()}

from sklearn.metrics import classification_report, confusion_matrix
cm_train = confusion_matrix(training_set.classes, y_pred)
print('Confusion Matrix')
print(cm_train)
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(training_set.classes, y_pred, target_names=target_names))

plt.figure(figsize=(8,8))
plt.imshow(cm_train, interpolation='nearest')
plt.colorbar()
tick_mark = np.arange(len(target_names))
_ = plt.xticks(tick_mark, target_names, rotation=90)
_ = plt.yticks(tick_mark, target_names)

### Confusion Matrix and Classification on test set

In [21]:
y_pred = fer_model.predict(test_set)
y_pred = np.argmax(y_pred, axis=1)
class_labels = test_set.class_indices
class_labels = {value:key for key,value in class_labels.items()}

#from sklearn.metrics import classification_report, confusion_matrix
cm_test = confusion_matrix(test_set.classes, y_pred)
print('Confusion Matrix')
print(cm_test)
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(test_set.classes, y_pred, target_names=target_names))

plt.figure(figsize=(8,8))
plt.imshow(cm_test, interpolation='nearest')
plt.colorbar()
tick_mark = np.arange(len(target_names))
_ = plt.xticks(tick_mark, target_names, rotation=90)
_ = plt.yticks(tick_mark, target_names)

In [22]:
emotions = {0:'angry',
            1:'disgust',
            2:'fear',
            3:'happy',
            4:'neutral',
            5:'sad',
            6:'surprise'}

path = '../input/emotor-test-set'

In [23]:
datagen = ImageDataGenerator(rescale=1/255.)

pred_generator = datagen.flow_from_directory(path,
                                             batch_size=64,
                                             class_mode=None,
                                             shuffle=False,
                                             target_size=(image_size, image_size),
                                             color_mode='grayscale',)

In [25]:
preds = fer_model.predict(pred_generator)
preds = tf.argmax(preds, axis=1).numpy()

In [26]:
images = os.listdir(os.path.join(path, 'EMOTOR_TEST'))
pred_dict = [{'img_name':img, 'label':emotions[pred]} for img, pred in zip(images, preds)]

In [27]:
import csv
with open('predictions.csv', 'w', newline='') as f:
    fieldnames=['img_name', 'label']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(pred_dict)

In [28]:
from IPython.display import FileLink
FileLink(r'./fer_model.h5')